# Build a WFlow model anywhere

In this notebook we will setup a WFlow model coupled to the FloodAdapt backend

## **Step 0:** Import dependencies

In [ ]:
from pathlib import Path
import hydromt
import geopandas as gpd
from hydromt.log import setuplog
from hydromt.config import configread
from hydromt_wflow import WflowModel

import sys
sys.path.append("/home/jovyan/work")
from DT_flood.utils.plot_utils import plot_wflow_model

In [ ]:
model_name = 'Humber'
model_path = Path('/home/jovyan/work/Project') / model_name
#model_path = Path("/home/wotromp/InterTwin/FloodAdapt_database") / model_name
# model_path = Path('c:/Repositories/DT-flood/FloodAdapt_database') / model_name
# model_path = Path("../FloodAdapt_database") / model_name

wf_root = model_path / Path('static/templates/wflow')

wf_logger_name = 'WFlow_Logger'

region_fn = Path('/home/jovyan/work/Data/region') / 'HumberDelta_large.geojson'
#region_fn = model_path / 'HumberDelta_large.geojson'

In [ ]:
# Use this for own data_catalog file
data_catalog_fn = Path('/home/jovyan/work/Data/data_catalog.yml')
#data_catalog_fn = Path("/home/wotromp/InterTwin/Data/Humber/data_catalog.yml")
# data_catalog_fn = Path('c:/Users/tromp_wm/OneDrive - Stichting Deltares/Documents/InterTwin Project/Data/DTFlood_test_catalog.yml')

# data_dict = {
#     'topo': 'copdem30',
#     'bathy': 'gebco',
#     'waterlevels': 'gtsm_hourly',
#     'meteo': 'era5_hourly',
#     'infiltration': 'gcn250',
#     'lulc': 'esa_worldcover',
#     'basins': 'BasinAtlas',
# }

In [ ]:
wf_logger = setuplog(wf_logger_name,log_level=10)

# data_catalog = hydromt.DataCatalog(data_libs=data_catalog_fn,logger=wf_logger)

# wf = WflowModel(data_libs=[data_catalog_fn, 'deltares_data=v2023.2'], root=wf_root, mode='w+')
wf = WflowModel(data_libs=[data_catalog_fn], root=wf_root, mode='w+')

In [ ]:
ini_fn = model_path / "wflow_build.ini"
opt = configread(ini_fn)

region = gpd.read_file(region_fn)

In [ ]:
opt

In [ ]:
wf.build(region={'basin': region, 'outlets': True}, opt=opt)

In [ ]:
gauges_fn = wf_root.parent/"overland"/"gis"/"src.geojson"
wf.setup_gauges(
    gauges_fn=gauges_fn,
    index_col='index',
    snap_to_river=True,
    snap_uparea=True,
    derive_subcatch=False,
)

In [ ]:
wf.geoms["gauges_src"]

In [ ]:
setting_toml = {
    "netcdf.path": f"output_scalar.nc",
    "netcdf.variable": [
        {
            "name": "Q_src",
            "map": "gauges_src",
            "parameter": "lateral.river.q_av",
        }
    ],
}
for option in setting_toml:
    wf.set_config(option, setting_toml[option])
wf.write_config()

In [ ]:
wf.write()

In [ ]:
wf_read = WflowModel(root=wf_root, mode="r")
wf_read.read()

In [ ]:
plot_wflow_model(wf_read)